# Week 10 (In Reality 9) - Amin's Class Session

## References:

* https://www.tutorialspoint.com/clojure/clojure_exception_handling.htm
* http://theatticlight.net/posts/Lazy-Sequences-in-Clojure/
* The Joy of Clojure - Chapters 6 and 2.9

## Part 1 - Structural Sharing

Clojure's persistent data structures allows for structural sharing, which is essentially eliminating the need for a brand new structure if one already exists in memory. To explain further, we will look at a very simple example of strucutural sharing using lists. 

Below is a base list named `pizzalist`, and two other lists named `pizza1` and `pizza2` which take `pizzalist` and add a new pizza to the front.

In [ ]:
(def pizzalist (list "Meatlover" "Pepperoni"))

(def pizza1 (cons "Hawaiian" pizzalist))
(def pizza2 (cons "Veggie" pizzalist))

pizza1
pizza2

;(= (next pizza1) (next pizza2))
;(identical? (next pizza1) (next pizza2))

So that's pretty cool, the nexts of `pizza1` and `pizza2` are not only equal, but they are the exact same object.

The features supported by lists are kind of limited, vectors and maps also provide structural sharing, while also allowing you to change values **anywhere** in the collection.

Let's build a simple tree to help demonstrate how a tree can allow interior changes and maintain shared structure at the same time. Each node will be a map with a *value*, a *left* branch, and a *right* branch, like so:

In [ ]:
{:val 5, :Left nil, :Right nil}

This is a very simple tree with a single node holding the value 5, with empty left and right branches.

Lets create an `add-to-tree` function to build up our tree, starting with an initial case of an empty tree:

In [ ]:
(defn add-to-tree [tree value]
  (cond
    (nil? tree) {:val value, :Left nil, :Right nil}))

(println (add-to-tree nil 5))

Simple enough right? Now lets handle the case where an item is being added to a **non-empty** tree:

In [ ]:
(defn add-to-tree [tree value]
  (cond
    (nil? tree)            {:val value, :Left nil, :Right nil}
    (< value (:val tree))  {:val (:val tree),
                            :Left (add-to-tree (:Left tree) value),
                            :Right (:Right tree)}))

(def tree1 (add-to-tree nil 5))
(def tree1 (add-to-tree tree1 3))
(def tree1 (add-to-tree tree1 2))

(println tree1)

Now we can handle insertion of values that are less than the root node, lets add the final condition for handling the insertion of values that are not less than the node value:

In [ ]:
(defn add-to-tree [tree value]
  (cond
    (nil? tree)            {:val value, :Left nil, :Right nil}
    (< value (:val tree))  {:val (:val tree),
                            :Left (add-to-tree (:Left tree) value),
                            :Right (:Right tree)}
    :else {:val (:val tree),
           :Left (:Left tree),
           :Right (add-to-tree (:R tree) value)}))


(def tree1 (add-to-tree nil 5))
(def tree1 (add-to-tree tree1 3))
(def tree1 (add-to-tree tree1 2))

(def tree2 (add-to-tree tree1 7))

(println "Tree 1 --> " tree1)
(println "Tree 2 --> " tree2)

(= (:Left tree1) (:Left tree2))
(identical? (:Left tree1) (:Left tree2))

We can see that although the 2 trees are different objects, both of their left subtrees are actually pointing to the exact same object. The figure below will visualize this tree.

<img src="https://dpzbhybb2pdcj.cloudfront.net/fogus2/Figures/06fig01.jpg" alt="drawing" width="400" align="left"/>

So this example fails when compared to Clojure's production-quality code:
* It can only store numbers
* It will overflow the stack if the tree gets too deep
* It can create unbalanced trees that have worst-case algorithmic complexity

Although structural sharing as described above using `add-to-tree` can reduce the memory footprint of persistent data structures, it alone is insufficient. Instead, Clojure leans heavily on the notion of lazy sequences to further reduce its memory footprint.

## Part 2 - Laziness

Programming languages are **eager** when arguments to functions are immediately evaluated when passed, and Clojure in most cases follows this pattern as well. Observe the following:

In [ ]:
(- 13 (+ 2 2))

The expression (+ 2 2) is eagerly evaluated, and its result 4 is passed on to the subtraction function. However, a **lazy** programming language such as Haskell (Hudak 2000) will evaluate a function argument only if that argument is needed in an overarching computation.

In the example below, we take a look at a simple function that resembles Clojure's built-in range function, with the only differences being that it creates a **lazy sequence** instead of a long range, and also it doesn't accept a **step** argument.

### Lazy Sequences

In [ ]:
(defn simple-range [i limit]
  (lazy-seq
    (when (< i limit)
      (cons i (simple-range (inc i) limit)))))

(println (range 0 10))
(println (simple-range 0 10))

(println (class (range 0 10)))
(println (class (simple-range 0 10)))

The figure below is a representation of the state of memory when the REPL has printed the first two items in a simple-range sequence, but it has not yet printed any other items. In each step, if the step is unrealized, then it will contain a function or closure of no arguments that can be called later to realize the step. Once it is realized, the step will be released as pictured in the first 2 steps.

<img src="https://dpzbhybb2pdcj.cloudfront.net/fogus2/Figures/06fig02.jpg" alt="drawing" width="400" align="left"/>

### Losing Your Head

The primary advantage of laziness in Clojure is that it prevents the full realization of interim results during a calculation. If somehow by accident you manage to hold on to the head of a sequence, then that sequence will be prevented from being garbage collected by Clojure. Retaining the head can be done by binding it to a local using a **let** or a **binding**.

In [ ]:
(let [r (range 1e8)]
  (first r)
  (last r))

;(let [r (range 1e8)]
;  (last r)
;  (first r))

Clojure's compiler can deduce that in the first example, the retention of r is no longer needed when the computation of `(last r)` occurs, and therefore Clojure aggressively clears it. However, in the second example, the head is still needed later in computation, and can no longer be safely cleared. The compiler could form some sort of rearranging, but it doesn't because it cannot guarantee that the current order is unimportant. 

### Infinite Sequences

Because Clojure's sequences are lazy, they have the potential to be infinitely long. Clojure provides a number of functions for generating and working with infinite sequences, such as `iterate`, `take`, `drop`, `take-while`, and `drop-while`.

The `take` function returns a sequence of the first n items in the sequence, it is mainly used as a **limiter** for a sequence. Interestingly, the `take` function always returns a `LazySeq`, even if the sequence being taken from is something else.

In [ ]:
(range 20)
;(class (range 20))

;(take 10 (range 20))
;(class (take 10 (range 20)))

The `iterate` function returns a lazy sequence of repetitions of the arguments that are passed to it. For example, `(iterate f x)` will return `[x, (f x), (f (f x)), ... ]`

In [ ]:
(take 20 (iterate inc 1))

;(take 20 (iterate (fn [n] (/ n 2)) 1))

The `drop` function returns a new sequence equivalent to the current sequence with the first n items removed.

In [ ]:
(drop 10 (range 20))

;(take 20 (drop 10 (iterate (fn [n] (/ n 2)) 1)))

The `take-while` function returns a sequence of items in a sequence that match a given condition.

In [ ]:
(take-while #(< % 25) (iterate inc 1))

;(take-while #(< % 5000) (iterate (fn [n] (* n 2)) 1))

The `drop-while` function returns a new sequence equivalent to the current sequence with the first n items that match a given condition.

In [ ]:
(take 20 (drop-while #(< % 25) (iterate inc 1)))

;(take 20 (drop-while #(< % 1000) (iterate (fn [n] (* n 2)) 1)))

Now, using the information above, can you guys implement a function that will print out the first 20 numbers in the **Fibonnaci sequence**?

The Fibonnaci sequence is a sequence where the current value in the sequence is equivalent to the last 2 values added together `[0, 1, 1, 2, 3, 5, 8, 13, ...]`

In [ ]:
;Paste code here

## Exceptions

Lets switch gears and talk a little bit about Clojure's facilities for handling exceptions. Clojure provides a couple of forms for throwing and catching runtime exceptions, `throw` and `catch`.

At any point, you can just throw an exception along with a message, and it will show that message. You may also wrap your code in a `try/catch` statement, in order to catch specific exceptions during computation.

In [ ]:
;(throw (Exception. "Houston, we have a problem."))

(try
  (/ 10 /)
  (catch ArithmeticException e "Stop dividing by zero bud.")
  (catch Exception e (println "You are so bad at Clojuring!"))
  ;(catch Exception e (println "You are so bad at Clojuring!") (.getMessage e))
  (finally (println "Returning... ")))

For a small exercise, try and create a function that takes a function as input, and performs the same exception handling as above.

In [ ]:
;Paste code here

## Errors

As you can see, Clojure's exception handling is very similar to Java. To catch errors, the syntax is exactly the same as above, except with **Error** instead of **Exception**.

In [ ]:
(try
  (throw (Error. "I done throwed in CLJS"))
  (catch Error err "I done catched in CLJS"))

## Difference Between Errors and Exceptions

The main difference between **Errors** and **Exceptions** is that errors are **irrecoverable**, meaning the program cannot recover from them. For example, errors include `OutOfMemoryError`, `VirtualMachineError`, `AssertionError`, etc. Exceptions on the other hand aren't quite so terminal, they include `ArithmeticException`, `NullPointerException`, `ArrayIndexOutOfBoundsException`, etc.

The following diagram shows how the hierarchy of exceptions in Clojure is organized. It’s all based on the hierarchy defined in Java.

<img src="https://www.tutorialspoint.com/clojure/images/exceptions_in_clojure.jpg" alt="drawing" width="550" align="middle"/>

## ALL DONE! Any questions?